In [1]:
import pandas as pd
import numpy as np
import os

import pickle
from collections import defaultdict

In [4]:
DATA_DIR = '../inbox'
DATA_CSV_DIR = '../data'
FILE_NAME = '04-1 仿真数据库样例数据V1.0.20170713.xlsx'
DATA_PATH = os.path.join(DATA_DIR, FILE_NAME)

sheet_names = \
    ['i_container_capacity',
    'i_equipment_io',
    'i_equipment_parameter',
    'i_equipment_resource',
    'i_queue_io',
    'i_reload_setting',
    'i_resource_limit', 
    'i_od_parcel_landside',
    'i_unload_setting',
    'i_reload_setting',]

if 1 == 1:
    tables = {name : pd.read_excel(DATA_PATH, sheetname=name) for name in sheet_names}
    for name, table in tables.items():
        table.to_csv(f'../data/{name}.csv', index=0)
else: 
    tables = {name : pd.read_csv(os.path.join(DATA_CSV_DIR, name + '.csv')) for name in sheet_names}

In [9]:
tables['i_queue_io']['sorter_zone'].apply(lambda x: x[:2]).unique()

array(['LM', 'LS', 'SE', 'AC', 'AM', 'AS', 'MS'], dtype=object)

# change
- queue io
- equipment io

In [ ]:
table_equipment_io = tables['i_equipment_io']

cols = ['equipment_port_last',
'equipment_port_next',
'equipment_type',
'queue_id',
'process_time',
'allocate_rule',]

table_queue_io = tables['i_queue_io'][cols]

import collections

def generate_tree_dict(table: pd.DataFrame):
    tree_dict = collections.defaultdict(set)
    for _, row in table.iterrows():
        tree_dict[row['equipment_port_last']].add(row['equipment_port_next'])
    return tree_dict

In [ ]:
table_queue_io['equipment_id_in'] = table_queue_io['queue_id'].str.split('_').str.get(0)
table_queue_io['equipment_id_out'] = table_queue_io['queue_id'].str.split('_').str.get(1)

MS_code = [f'u{x}' for x in range(1, 9)]

table_package = table_queue_io[~table_queue_io.equipment_id_out.isin(MS_code)]

In [ ]:
tree_package = generate_tree_dict(table_package)

In [ ]:
import copy
from itertools import product

In [ ]:
tree_dict = copy.deepcopy(tree_package)

In [ ]:
def clean_h(locs: list):
    return [x for x in locs if 'h' not in x]

def check_h(locs: list):
    return any([x for x in locs if 'h' in x])

def generate_path(change_paths: list, fixed_paths: list):
    
    pathss = []
    end_locs = []
    
    for path in change_paths:
                
        last_loc = path[-1]
        new_locs = list(tree_dict[last_loc])
                
        if check_h(path):
            new_locs = clean_h(new_locs)
                    
        if new_locs:
            paths = [path + [new_loc] for new_loc in new_locs]
            pathss.extend(paths)
        else:
            fixed_paths.append(path)
        
        end_locs.append(new_locs)

    if any(end_locs):
        return generate_path(pathss, fixed_paths)
    else:
        return set(tuple(x) for x in (change_paths + fixed_paths))

In [ ]:
all_paths = generate_path([['r1_4']], [])

In [ ]:
len(all_paths)

In [ ]:
all_paths

In [ ]:
import random
from functools import partial

In [ ]:
def path_filter(path, start, end):
    return (path[0] == start) and (path[-1] == end)

path_filter_test1 = partial(path_filter, start='r1_1', end='c1_14')
path_filter_test2 = partial(path_filter, start='r1_1', end='c1_16')

In [ ]:
%time path_test1 = list(filter(lambda x: path_filter_test1(x), all_paths))
%time path_test2 = list(filter(lambda x: path_filter_test2(x), all_paths))

In [ ]:
test1 = set(tuple(x) for x in path_test1)
test2 = set(tuple(x) for x in path_test2)

In [ ]:
test1

In [ ]:
test2

In [ ]:
list(filter(lambda x: check_h(x), all_paths))